In [6]:
import numpy as np
import scipy as sp
from scipy.linalg import solve_banded, solve
import time

In [34]:
k = 100
n = 2000
delta = 0.5
eta = 2

du = np.ones(n) * -1
dm = np.ones(n) * 2
dm[0] = 1
dm[-1] = 1
dl = np.ones(n) * -1
DTD = np.vstack((du, dm, dl))
delta_DTD_dense = (
    delta * sp.sparse.diags([DTD[0, 1:], DTD[1, :], DTD[2, :-1]], offsets=[1, 0, -1])
).toarray()

A = np.random.randn(k, n)
b = np.random.randn(k)
c = A.T @ b

Generic method (dense inverse).

Flop count: $O(n^3 + 2kn^2)$


In [40]:
time_start = time.perf_counter()
x_generic = solve(A.T @ A + delta_DTD_dense + eta * np.eye(n), c)
time_end = time.perf_counter()
time_generic = time_end - time_start
print(f"Solution to the system: {x_generic}")
print(f"Generic method time: {time_generic:.6f} seconds")

Solution to the system: [ 0.0036727   0.00152743  0.00285381 ... -0.00298233  0.00047727
  0.00687566]
Generic method time: 0.395068 seconds


Efficient method relying on Sherman-Morrison-Woodbury and banded inverse.

Flop count: $O(k^2 n + k^3)$. Much better when $k \ll n$.


In [41]:
time_start = time.perf_counter()
T = delta * DTD
T[1, :] += eta
d = solve_banded((1, 1), T, c)
E = solve_banded((1, 1), T, A.T)
F = np.eye(k) + A @ E
g = solve(F, A @ d)
h = solve_banded((1, 1), T, A.T @ g)
x_eff = d - h
time_end = time.perf_counter()
time_eff = time_end - time_start
print(f"Solution to the system: {x_eff}")
print(f"Efficient method time: {time_eff:.6f} seconds")

assert np.allclose(x_generic, x_eff)
print("Solutions match!")
print("Time savings: {:.2f}x".format(time_generic / time_eff))

Solution to the system: [ 0.0036727   0.00152743  0.00285381 ... -0.00298233  0.00047727
  0.00687566]
Efficient method time: 0.006030 seconds
Solutions match!
Time savings: 65.51x
